<a href="https://colab.research.google.com/github/chris-iscool/machinelearning/blob/main/Ising_TC_Monte_Carlo_and_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import numpy.random as random
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from collections import deque

!pip install array2gif
from array2gif import write_gif

# 2D Ising Model

Hamiltonian:

$$
\begin{align}
    \mathcal{H} & = -J \sum_{<i, j>} s_i s_j
\end{align}
$$

Critical temperature according to exact Onsager solution:

$$
\begin{align}
    T_c & = \frac{2 J}{\ln(1 + \sqrt{2})}
\end{align}
$$

For now, assume periodic boundary conditions!

# Metropolis Algorithm

*   Start with a randomly initialized spin configuration $\{s_i\}$
*   Propose a random spin-flip 
*   Calculate energy difference between new configuration and old, $\Delta H = E' - E$
*   Randomly accept spin flip according to Metropolis: $p = \min(1, e^{-\beta \Delta E})$ 
*   Take a snapshot every n flips (eg every 5 flips)

# Swendsen-Wang Algorithm

*   Bonds_h[i, j] between spins[i, j] and spins[i, j+1]
*   Bonds_v[i, j] between spins[i, j] and spins[i+4, j]
*   Update step for Swendsen-Wang:
*   For all NN pairs, check if $s_i s_j = +1$, if so, create bond with probability
$$
p = 1 - \exp(-2 \beta J)
$$
*   Recursive algorithm to identify clusters, ie. magnetic domains
*   For all identified clusters, flip with probability $q = \frac{1}{2}$
*   Bonds are recreated in each step

UPDATE: Will now merge finding and flipping of clusters into one function.

Use pseudo code of Wende and Steinke



In [ ]:
J = 1.0

# need to implement Ising Lattice as a class, only way to do this without sending the lattice as arguments the whole time
class IsingLattice:
    def __init__(self, L_x=8, L_y=8, T=1.0):
        self.L_x = L_x
        self.L_y = L_y
        self.beta = 1/T

        # Step 1: Randomly initialize spins
        self.spins = random.choice([-1, 1], (L_y, L_x))
        self.in_cluster = np.zeros((L_y, L_x))
        self.history = None

    # New Version: Need much less overhead, just calculate dH of spin flip directly
    def dE(self, i, j):
        E, E_flip = 0., 0.

        for dy, dx in ((-1, 0), (1, 0), (0, -1), (0, 1)):
            yy = (i + dy) % self.L_y
            xx = (j + dx) % self.L_x
            E -= J*self.spins[yy, xx] * self.spins[i, j]
            E_flip += J*self.spins[yy, xx] * self.spins[i, j]
        
        return E_flip - E

    def run_metropolis(self, t_total=10000, batch_size=1):
        N = t_total // batch_size
        self.history = np.zeros((N, self.L_y, self.L_x))
        self.history[0] = self.spins

        for t in range(t_total):
            # Step 2: propose batch_size spin flips
            random_y = random.randint(0, self.L_y)
            random_x = random.randint(0, self.L_x)

            # Step 3: Calculate energy difference of proposed flips. This can be done locally.
            dH = self.dE(random_y, random_x)
            
            # Step 4: Calculate probability of flip
            p = min([1., np.exp(-self.beta * dH)])

            # Step 5: Do the flip
            r = random.random()
            if r <= p:
                self.spins[random_y, random_x] *= -1

            if (t % batch_size) == 0:
                self.history[t // batch_size] = self.spins
    
        return self.spins
    
    def find_and_flip_cluster(self, y0, x0):
        flip = False
        r = random.random()
        # p_(flip cluster) = 0.5
        if r < 0.5:
            flip = True
        
        queue = deque([(y0, x0)])
        self.in_cluster[y0, x0] = 1

        while queue:
            y, x = queue.pop()
            for dy, dx in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                yy = (y + dy) % self.L_y
                xx = (x + dx) % self.L_x
                if self.in_cluster[yy, xx] == 1:
                    continue
                if self.spins[y, x] * self.spins[yy, xx] == -1:
                    continue
                
                # p_(create bond) = 1 - exp(-2 J beta)
                r = random.random()
                if r < (1 - np.exp(-2*J*self.beta)):
                    queue.append((yy, xx))
                    self.in_cluster[yy, xx] = 1
                
            if flip:
                self.spins[y, x] *= (-1)
        
        queue.clear()
    
    def swendsen_update(self):
        for y in range(self.L_y):
            for x in range(self.L_x):
                if self.in_cluster[y, x] == 1:
                    continue
                else:
                    self.find_and_flip_cluster(y, x)
        
        # if you want to measure stuff like magnetization, do it here

        # Reset all bonds as unmarked
        self.in_cluster = np.zeros((self.L_y, self.L_x))
    
    # The helper functions should do the heavy lifting, this should be easy!
    def run_swendsen(self, t_total=1000):
        self.history = np.zeros((t_total, self.L_y, self.L_x))

        for t in range(t_total):
            self.swendsen_update()
            self.history[t] = self.spins

        return self.spins

In [ ]:
def plot_magnet(magnet, fps=25):
    history = magnet.history
    t_total = history.shape[0]
    interval = 1000 / fps
    
    fig = plt.figure(figsize=(5, 5))
    im = plt.imshow(history[0])

    def animate(i):
        im.set_array(history[i])

        return [im]
    
    my_ani = ani.FuncAnimation(fig, animate, frames=range(t_total), interval=interval, repeat=True)
    return my_ani


# ising = IsingLattice(25, 25, 0.1)
# spins = ising.run(20000, 5)

# cool_ani = plot_magnet(ising, 50)
# cool_ani

In [ ]:
# Since matplotlib animations are kinda wonky on colab, also build a quick and dirty gif maker
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
def color_lattice(lattice):
    blue = np.ones(lattice.shape, dtype=np.int) * 153
    red = np.ones(lattice.shape, dtype=np.int) * 76
    red[lattice < 0] = 153
    blue[lattice < 0] = 0
    green = np.zeros(lattice.shape, dtype=np.int)
    return np.array([red, green, blue])

def ising_to_gif(history, filename, fps=25):
    print('Frames: {}'.format(history.shape[0]))
    colors = []

    write_gif([color_lattice(lattice) for lattice in history], filename, fps=fps)

In [ ]:
ising = IsingLattice(50, 50, 0.5)
spins = ising.run_metropolis(50000, 5)

ising2 = IsingLattice(50, 50, 0.5)
spins2 = ising2.run_swendsen(50)

ising_to_gif(ising.history, '/drive/My Drive/Colab Notebooks/ising.gif')
ising_to_gif(ising2.history, '/drive/My Drive/Colab Notebooks/ising2.gif', 1)

0.0
0.02
0.04
0.06
0.08
0.1
0.12
0.14
0.16
0.18
0.2
0.22
0.24
0.26
0.28
0.3
0.32
0.34
0.36
0.38
0.4
0.42
0.44
0.46
0.48
0.5
0.52
0.54
0.56
0.58
0.6
0.62
0.64
0.66
0.68
0.7
0.72
0.74
0.76
0.78
0.8
0.82
0.84
0.86
0.88
0.9
0.92
0.94
0.96
0.98
Frames: 10000
Frames: 50


# Classification with Neural Nets Part 1 - Supervised training

In [ ]:
import tensorflow as tf

In [ ]:
# Idea: for fixed lattice sites, generate about 100 datasets below critical T and 100 above
# Double the data sets by using the Z2 symmetry
# Train a simple MLP on the data, see how it performs
T_c = 2*J / (np.log(1 + np.sqrt(2)))
x_dim = 50
y_dim = 50

def generate_random_data(L_x, L_y, N):
    T_ordered = random.rand(N) * (T_c - 0.1) + 0.1 
    T_disordered = random.rand(N) * T_c + T_c 

    x = np.zeros((4*N, L_y, L_x))
    y = np.zeros(4*N, dtype=np.int)
    
    for i in range(N):
        lattice = IsingLattice(L_x, L_y, T_ordered[i])
        x[i] = lattice.run_swendsen(25)
        y[i] = 1 # y = 1: ordered phase
    
    for i in range(N):
        lattice = IsingLattice(L_x, L_y, T_disordered[i])
        x[i+N] = lattice.run_swendsen(25)
        y[i+N] = 0 # y = 0: disordered phase
    
    # Z2 symmetry -> double dataset
    for i in range(2*N):
        x[i+2*N] = x[i] * (-1)
        y[i+2*N] = y[i]

    indices = np.arange(0, 4*N)
    random.shuffle(indices)
    x = x[indices]
    y = y[indices]    
    return x, y

x, y = generate_random_data(x_dim, y_dim, 100)
x = x[:, :, :, np.newaxis]
print(x.shape)
print(y.shape)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.76
0.8
0.84
0.88
0.92
0.96
0.0
0.04
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.76
0.8
0.84
0.88
0.92
0.96
0.0
0.04
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.76
0.8
0.84
0.88
0.92
0.96
0.0
0.04
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.76
0.8
0.84
0.88
0.92
0.96
0.0
0.04
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.76
0.8
0.84
0.88
0.92
0.96
0.0
0.04
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.76
0.8
0.84
0.88
0.92
0.96
0.0
0.04
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.76
0.8
0.84
0.88
0.92
0.96
0.0
0.04
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.76
0.8
0.

In [ ]:
# Wow, that was fast... Okay, lets define a model

ising_mlp = tf.keras.Sequential([
                                 tf.keras.layers.Conv2D(64, (5, 5), (2, 2), activation='relu', input_shape=(x_dim, y_dim, 1)),
                                 tf.keras.layers.MaxPooling2D((2, 2)),
                                 tf.keras.layers.Conv2D(128, (5, 5), (2, 2), activation='relu'),
                                 tf.keras.layers.MaxPooling2D((2, 2)),
                                 tf.keras.layers.Flatten(),
                                 tf.keras.layers.Dense(256, activation='relu'),
                                 tf.keras.layers.Dense(2, activation='softmax')
])

ising_mlp.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

In [ ]:
ising_mlp.fit(x, y, epochs=20, validation_split=0.2, verbose=1)

Epoch 1/20
10/10 [==============================] - 1s 86ms/step - loss: 0.3089 - accuracy: 0.8562 - val_loss: 0.0204 - val_accuracy: 1.0000
Epoch 2/20
10/10 [==============================] - 1s 63ms/step - loss: 0.0350 - accuracy: 0.9906 - val_loss: 0.1456 - val_accuracy: 0.9875
Epoch 3/20
10/10 [==============================] - 1s 64ms/step - loss: 0.1307 - accuracy: 0.9719 - val_loss: 0.0424 - val_accuracy: 0.9875
Epoch 4/20
10/10 [==============================] - 1s 62ms/step - loss: 0.0406 - accuracy: 0.9844 - val_loss: 0.1046 - val_accuracy: 0.9875
Epoch 5/20
10/10 [==============================] - 1s 61ms/step - loss: 0.0150 - accuracy: 0.9937 - val_loss: 0.0655 - val_accuracy: 0.9750
Epoch 6/20
10/10 [==============================] - 1s 61ms/step - loss: 0.0308 - accuracy: 0.9906 - val_loss: 0.0209 - val_accuracy: 0.9875
Epoch 7/20
10/10 [==============================] - 1s 61ms/step - loss: 0.0140 - accuracy: 0.9906 - val_loss: 0.0751 - val_accuracy: 0.9750
Epoch 8/20
10

In [ ]:
test_x, test_y = generate_random_data(x_dim, y_dim, 200)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
0.0
0.04
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.76
0.8
0.84
0.88
0.92
0.96
0.0
0.04
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.76
0.8
0.84
0.88
0.92
0.96
0.0
0.04
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.76
0.8
0.84
0.88
0.92
0.96
0.0
0.04
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.76
0.8
0.84
0.88
0.92
0.96
0.0
0.04
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.76
0.8
0.84
0.88
0.92
0.96
0.0
0.04
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.76
0.8
0.84
0.88
0.92
0.96
0.0
0.04
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.76
0.8
0.84
0.88
0.92
0.96
0.0
0.04
0.08
0.12
0.16
0.2
0.24
0.28
0.32
0.36
0.4
0.44
0.48
0.52
0.56
0.6
0.64
0.68
0.72
0.

In [ ]:
test_loss, test_acc = ising_mlp.evaluate(test_x, test_y, verbose=2)

25/25 - 0s - loss: 0.0591 - accuracy: 0.9825 - 406ms/epoch - 16ms/step


# Summary

It seems that a simple MLP is not able to accurately predict the phase of a given Ising configuration, however a simple CNN is able to make very accurate predictions.

# Restricted Boltzmann Machines

arXiv:1810.11503v1
arXiv:1810.08179v1 

# Auto encoders

https://doi.org/10.1140/epjb/e2020-100506-5 

Strategy: Use a simple auto encoder model (cf. reference) to extract the "net magnetization" of a given spin state and thus predict with unsupervised learning if the lattice has thermalized into an ordered or unordered state.

In [ ]:
# I'm not sure yet what to do with it, but here is the model definition of the auto encoder, using the same dimensions as in the paper
# The encoder will down sample the full lattice to a single logit z (latent variable)
# The decoder will up sample the single logit back to the original size

from tensorflow.keras.models import Model

class AutoEncoder(Model):
    def __init__(self, L_y, L_x):
        super().__init__()
        self.encoder = tf.keras.Sequential([
                                            tf.keras.layers.Flatten(input_shape=(L_y, L_x, 1)),
                                            tf.keras.layers.Dense(625, activation='relu'),
                                            tf.keras.layers.Dropout(0.3),
                                            tf.keras.layers.Dense(256, activation='relu'),
                                            tf.keras.layers.Dropout(0.3),
                                            tf.keras.layers.Dense(64, activation='relu'),
                                            tf.keras.layers.Dropout(0.3),
                                            tf.keras.layers.Dense(1, activation='tanh')
        ])
        self.decoder = tf.keras.Sequential([
                                            tf.keras.layers.Dense(64, activation='relu', input_shape=(1,)),
                                            tf.keras.layers.Dropout(0.3),
                                            tf.keras.layers.Dense(256, activation='relu'),
                                            tf.keras.layers.Dropout(0.3),
                                            tf.keras.layers.Dense(625, activation='relu'),
                                            tf.keras.layers.Dense(L_y * L_x, activation=None),
                                            tf.keras.layers.Reshape((L_y, L_x, 1))
        ])
    
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
    def encode(self, x):
        encoded = self.encoder(x)
        return encoded

auto_encoder = AutoEncoder(50, 50)
auto_encoder.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy'])

In [ ]:
auto_encoder.fit(x, x, epochs=20, validation_split=0.2, verbose=1)

Epoch 1/20
10/10 [==============================] - 4s 170ms/step - loss: 0.9897 - accuracy: 0.0000e+00 - val_loss: 0.9349 - val_accuracy: 0.0000e+00
Epoch 2/20
10/10 [==============================] - 1s 89ms/step - loss: 0.8381 - accuracy: 0.1103 - val_loss: 0.6715 - val_accuracy: 0.2787
Epoch 3/20
10/10 [==============================] - 1s 73ms/step - loss: 0.7368 - accuracy: 0.2514 - val_loss: 0.6411 - val_accuracy: 0.2904
Epoch 4/20
10/10 [==============================] - 1s 79ms/step - loss: 0.7337 - accuracy: 0.2812 - val_loss: 0.6604 - val_accuracy: 0.2987
Epoch 5/20
10/10 [==============================] - 1s 87ms/step - loss: 0.7278 - accuracy: 0.2525 - val_loss: 0.6397 - val_accuracy: 0.2855
Epoch 6/20
10/10 [==============================] - 1s 87ms/step - loss: 0.7248 - accuracy: 0.2835 - val_loss: 0.6424 - val_accuracy: 0.2907
Epoch 7/20
10/10 [==============================] - 1s 96ms/step - loss: 0.7191 - accuracy: 0.2692 - val_loss: 0.6494 - val_accuracy: 0.2860
Epoc

In [ ]:
def generate_paper_data(L_x, L_y, T_min, T_max, dT, N_per_T):
    T = np.arange(T_min, T_max, step=dT)
    x = np.zeros((N_per_T * len(T), L_y, L_x))
    
    for i in range(len(T)):
        for j in range(N_per_T):
            lattice = IsingLattice(L_x, L_y, T[i+j])
            x[i] = lattice.run_metropolis(10000, 50) # it would be nice to investigate on achieving thermalization faster, like in the paper
   
    return T, x

In [ ]:
temperatures = np.linspace(1.0, 3.0, 100)

for t in range(len(temperatures)):
    ising = IsingLattice(500, 500, temperatures[t])
    ising.run_swendsen(20)

    filename = '/drive/My Drive/Colab Notebooks/ising/ising L100 {}.gif'.format(t)
    ising_to_gif(ising.history, filename, 1)


0.0
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95
Frames: 20
0.0
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95
Frames: 20
0.0
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95
Frames: 20
0.0
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95
Frames: 20
0.0
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95
Frames: 20
0.0
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95
Frames: 20
0.0
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95
Frames: 20
0.0
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95
Frames: 20
0.0
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95
Frames: 20
0.0
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95
F